In [1]:
import pandas as pd
import gensim
import nltk
import gensim.corpora as corpora
from gensim.models import LdaModel, CoherenceModel, Phrases
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import re

In [2]:
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\owner\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\owner\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import pandas as pd
data = pd.read_csv('kiva_train.csv')

In [4]:
data.head()

,id,country,en,gender,loan_amount,nonpayment,sector,status
0,1,Ecuador,<h4>Business Description</h4> \r\n <p> Don Mau...,M,825,lender,Food,1
1,2,Dominican Republic,Rosa Iris is a brilliant entrepreneur who sell...,F,450,partner,Retail,0
2,3,Kenya,Sirote is married with six children. Two of he...,F,600,lender,Agriculture,1
3,4,Kenya,David Mwangi Kimani is 33 years old and marri...,M,650,lender,Food,1
4,5,Dominican Republic,Nilda is a very persistent woman who has learn...,F,325,partner,Food,0


In [5]:
data.shape

(5454, 8)

In [6]:
# Initialize stopwords and lemmatizer
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

# 1. Preprocessing Function (Simplified with Bigram within the function)
def preprocess(text):
    if not isinstance(text, str):
        text = ''
    
    # Remove special characters and numbers
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Convert to lowercase and tokenize
    tokens = simple_preprocess(text.lower(), deacc=True)
    
    # Lemmatize and remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    # Create bigrams
    bigram = Phrases([tokens], min_count=5, threshold=100)
    tokens = bigram[tokens]
    
    return tokens

In [7]:
# Apply preprocessing
data['processed_en'] = data['en'].apply(preprocess)

In [8]:
print(data['en'][0])

<h4>Business Description</h4> 
 <p> Don Mauro co-owns a restaurant alongside his wife; it is located at an excellent location in one of the principal streets of his neighborhood.  The restaurant is open for breakfast, lunch, and dinner and also sells empanadas [corn cakes]. </p>

<h4>Loan Use</h4> 
<p> He will use the loan proceeds to better his restaurant by buying more fixed assets such as: chairs, tables and other furniture as needed. </p>

<h4> Personal Information</h4> 
<p> He is 49 years old, he is married and has three sons, his house is built of cement. His principal goal is to grow his business by reinvesting the revenue obtained. </p>




 
<p> <b>Translated from Spanish by Felipe Salcedo, Kiva Volunteer.</b><p>


In [9]:
print(data['processed_en'][0])

['business', 'description', 'mauro', 'co', 'owns', 'restaurant', 'alongside', 'wife', 'located', 'excellent', 'location', 'one', 'principal', 'street', 'neighborhood', 'restaurant', 'open', 'breakfast', 'lunch', 'dinner', 'also', 'sell', 'empanadas', 'corn', 'cake', 'loan', 'use', 'use', 'loan', 'proceeds', 'better', 'restaurant', 'buying', 'fixed', 'asset', 'chair', 'table', 'furniture', 'needed', 'personal', 'information', 'year', 'old', 'married', 'three', 'son', 'house', 'built', 'cement', 'principal', 'goal', 'grow', 'business', 'reinvesting', 'revenue', 'obtained', 'translated', 'spanish', 'felipe', 'salcedo', 'kiva', 'volunteer']


In [10]:
# Create Dictionary and Corpus
dictionary = corpora.Dictionary(data['processed_en'])
# Filter out extremes
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in data['processed_en']]

In [11]:
# Build LDA Model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=5, 
                     random_state=100,
                     update_every=1,
                     chunksize=4000,
                     passes=10,
                     alpha='auto',
                     eta='auto',
                     per_word_topics=True)

In [12]:
# 4. Print the 25 most significant terms per topic
for i in range(5):
    print(f'Topic {i+1}:')
    print([term for term, _ in lda_model.show_topic(i, topn=25)])
    print('\n')

Topic 1:
['u', 'school', 'family', 'married', 'able', 'buy', 'two', 'income', 'also', 'stock', 'one', 'purchase', 'pay', 'market', 'need', 'expand', 'fee', 'started', 'shop', 'three', 'requesting', 'time', 'customer', 'increase', 'due']


Topic 2:
['group', 'community', 'life', 'small', 'hope', 'entrepreneur', 'woman', 'one', 'esperanza', 'buy', 'clothing', 'family', 'husband', 'able', 'support', 'new', 'help', 'selling', 'customer', 'two', 'store', 'good', 'many', 'like', 'product']


Topic 3:
['rice', 'farmer', 'land', 'baba', 'sector', 'area', 'also', 'crop', 'part', 'bank', 'water', 'program', 'rural', 'communal', 'people', 'mifex', 'many', 'santa', 'month', 'lucia', 'farm', 'finance', 'association', 'production', 'use']


Topic 4:
['home', 'product', 'work', 'kiva', 'help', 'mifex', 'able', 'store', 'also', 'client', 'buy', 'use', 'husband', 'house', 'volunteer', 'translated', 'spanish', 'family', 'order', 'school', 'information', 'selling', 'started', 'make', 'org']


Topic 5:
['

In [13]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics using pyLDAvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)

# Display the pyLDAvis visualization
pyLDAvis.display(vis)

In [14]:
# Get the topic distribution for each document
topic_distributions = []
for bow in corpus:
    topic_distribution = lda_model.get_document_topics(bow, minimum_probability=0)
    topic_distributions.append([weight for _, weight in topic_distribution])

# Create a DataFrame
topic_df = pd.DataFrame(topic_distributions, columns=[f'Topic_{i+1}' for i in range(lda_model.num_topics)])

# Add the processed text to the DataFrame
topic_df.insert(0, 'Processed_Text', data['processed_en'])

# Add a "Dominant_Topic" column with the name of the topic with the highest weight
topic_df['Dominant_Topic'] = topic_df.iloc[:, 1:].idxmax(axis=1)

topic_df

,Processed_Text,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Dominant_Topic
0,"[business, description, mauro, co, owns, resta...",0.001872,0.001627,0.000485,0.994561,0.001455,Topic_4
1,"[rosa, iris, brilliant, entrepreneur, sell, si...",0.001600,0.521333,0.000414,0.001062,0.475591,Topic_2
2,"[sirote, married, six, child, two, child, marr...",0.562832,0.001348,0.000402,0.001029,0.434390,Topic_1
3,"[david, mwangi, kimani, year, old, married, on...",0.906605,0.001690,0.000503,0.001289,0.089912,Topic_1
4,"[nilda, persistent, woman, learned, strive, be...",0.001741,0.995301,0.000451,0.001154,0.001353,Topic_2
...,...,...,...,...,...,...,...
5449,"[mary, kanyi, married, five, child, attended, ...",0.101405,0.001596,0.000475,0.001218,0.895305,Topic_5
5450,"[irene, general, store, sell, clothes, school,...",0.001156,0.029062,0.000299,0.968584,0.000899,Topic_4
5451,"[kevin, year, old, orphan, lost, parent, hiv, ...",0.887215,0.107542,0.000799,0.002046,0.002398,Topic_1
5452,"[business, description, last, twelve, year, se...",0.070323,0.001122,0.000334,0.927217,0.001003,Topic_4


In [15]:
topic_df['Dominant_Topic'].value_counts()

Dominant_Topic
Topic_1    1861
Topic_2    1417
Topic_4    1279
Topic_5     772
Topic_3     125
Name: count, dtype: int64